<a href="https://colab.research.google.com/github/gokulanv/ToyFederatedLearning/blob/master/DifferentialPrivacy/ToyDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toy Differential Privacy - Simple Database Queries

In this section I understand Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy

##First We Create a Simple Database
Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([ True, False,  True,  ..., False, False,  True])

# Project: Generate Parallel Databases
Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed.

In this first project, I create a list of every parallel database to the one currently contained in the "db" variable. Then, I create a function which both:


*   creates the initial database (db)
*   creates all parallel databases

In [0]:
db = torch.rand(num_entries) > 0.5
db = db.int()

In [3]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:])).int()
get_parallel_db(db, 10)

tensor([0, 0, 1,  ..., 1, 1, 0], dtype=torch.int32)

In [4]:
len(db), len(get_parallel_db(db, 100))

(5000, 4999)

In [0]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

In [0]:
pdbs = get_parallel_dbs(db)

In [0]:
def create_db_and_parallels(num_entries):
    
    db = (torch.rand(num_entries) > 0.5).int()
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(20)

# Lesson: Towards Evaluating The Differential Privacy of a Function
Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the maximum amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database.

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [0]:
db, pdbs = create_db_and_parallels(5000)

In [0]:
def query(db):
    return db.sum()

In [0]:
full_db_result = query(db)

In [0]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [13]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function
In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [0]:
# try this project here!
db, pdbs = create_db_and_parallels(5000)

In [0]:
def query_mean(db):
    return db.float().mean()


In [0]:
db_res = query_mean(db)

In [17]:
sensitivity =0 
for pdb in pdbs:
    
    pdb_result = query_mean(pdb)
    db_distance = torch.abs(pdb_result - db_res)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

sensitivity

tensor(0.0001)

"Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold
In this first project, I calculate the sensitivty for the "threshold" function.

First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
Then, I create databases of size 10 and threshold of 5 and calculate the sensitivity of the function.
Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [0]:
def sum(db ,threshold):
    return (db.sum() >threshold ).float()

In [0]:
db, pdbs = create_db_and_parallels(10)

In [0]:
def calc_sensitivity(n_entries):
    db, pdbs = create_db_and_parallels(n_entries)
    sensitivity = 0
    for pdb in pdbs:
        db_sum_result = sum(db, 5)
        pdb_response = sum(pdb, 5)
        distance = torch.abs(pdb_response - db_sum_result)
        if(distance > sensitivity):
            sensitivity = distance
    return(sensitivity)

In [21]:
for i in range(10):
    sens_calculated = calc_sensitivity(10)
    print(sens_calculated)

0
0
0
0
tensor(1.)
0
0
0
0
tensor(1.)


# Lesson: A Basic Differencing Attack
Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 5
In this project, I construct a database and then demonstrate how we can use two different sum queries to explose the value of the person represented by row 10 in the database

In [0]:
db, pdbs = create_db_and_parallels(36)

In [0]:
pdb = get_parallel_db(db, remove_index=5)

In [24]:
db[5]

tensor(0, dtype=torch.int32)

In [25]:
pdb

tensor([1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0], dtype=torch.int32)

In [26]:
# Attack using sum query
db.sum() - pdb.sum()

tensor(0)

In [27]:
sum(db, 7) - sum(pdb, 5)

tensor(0.)

In [28]:
# Attack using mean query
db.sum().float()/len(db) - pdb.sum().float()/len(pdb)

tensor(-0.0127)

If the database represents whether the person has cancer or not, now we can easily know that he has cancer using this differential attack.

## Project: Local Differential Privacy
As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)
Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

Flip a coin 2 times. \\
If the first coin flip is heads, answer honestly \\
If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)! \\

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained.

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results.

Let's implement this local DP for our database before!

In [0]:
db, pdbs = create_db_and_parallels(100)

In [30]:
db

tensor([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1], dtype=torch.int32)

In [0]:
first_coin_flip = (torch.rand(len(db)) > 0.5).float()
second_coin_flip = (torch.rand(len(db)) > 0.5).float()

In [0]:
# first honest response
res1 = db.float() * first_coin_flip

# second random response
res2 = (1 - first_coin_flip) * second_coin_flip

# LDP adds noise to the DB entries
augmented_db = res1 + res2

In [33]:
true_mean = torch.mean(db.float())
augmented_mean = torch.mean(augmented_db.float())
true_mean, augmented_mean

(tensor(0.5400), tensor(0.4900))

In [0]:
# Second response is always skewed by 50%
# So, we deskew the results
db_result = torch.mean(augmented_db).float() - 2 * 0.5

In [0]:
def query(db):
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    # first honest response
    res1 = db.float() * first_coin_flip

    # second random response
    res2 = (1 - first_coin_flip) * second_coin_flip

    # LDP adds noise to the DB entries
    augmented_db = res1 + res2
    true_mean = torch.mean(db.float())
    augmented_mean = torch.mean(augmented_db.float())
    db_result = augmented_mean * 2 - 0.5 # deskew

    return db_result, true_mean


### LDP Across different data points

In [36]:
db, pdbs = create_db_and_parallels(10)
query(db)

(tensor(0.9000), tensor(0.5000))

In [37]:
db, pdbs = create_db_and_parallels(100)
query(db)

(tensor(0.7200), tensor(0.5400))

In [38]:
db, pdbs = create_db_and_parallels(1000)
query(db)

(tensor(0.4660), tensor(0.5070))

In [39]:
db, pdbs = create_db_and_parallels(10000)
query(db)

(tensor(0.4814), tensor(0.4925))

## Varying Amounts of Noise
Here, I augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. 

In [0]:
def query(db, noise =0.2):
    actual_result = torch.mean(db.float())
    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    noisy_result = (db.float()* first_coin_flip) + ((1-first_coin_flip )* second_coin_flip)
    corrected_db_result = ((torch.mean(noisy_result.float()) /noise) - 0.5)* noise/(1-noise)
    return actual_result, corrected_db_result

In [41]:
db2,pdbs = create_db_and_parallels(5000)
actual_result, noisy_result = query(db2)

print("actual result" , actual_result)
print("noisy result", noisy_result)

db2,pdbs = create_db_and_parallels(10000)
actual_result, noisy_result = query(db2)

print("actual result" , actual_result)
print("noisy result", noisy_result)

actual result tensor(0.5102)
noisy result tensor(0.5125)
actual result tensor(0.5030)
noisy result tensor(0.5055)


## Lesson: How To Add Noise for Global Differential Privacy

This method adds noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid.


The amount of noise necessary to add to the output of a query is a function of four things:

*  the type of noise (Gaussian/Laplacian)
*  the sensitivity of the query/function
*  the desired epsilon (ε)
*  the desired delta (δ) \\

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

Querying Repeatedly
if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).


In [42]:
epsilon=0.5
import numpy as np

db, pdbs = create_db_and_parallels(100)

def sum_query(db):
     return   db.sum()

def laplacian_mechanism(db, query, sensitivity):
    beta = sensitivity/epsilon
    noise = torch.tensor(np.random.laplace(0,beta,1))
    return query(db) + noise

print(laplacian_mechanism(db,sum_query,1))

def mean_query(db):
     return torch.mean(db.float())

print(laplacian_mechanism(db,mean_query,0.01))

tensor([49.1228], dtype=torch.float64)
tensor([0.5323], dtype=torch.float64)


## An Example Scenario: A Health Neural Network
Consider a scenario - I work for a hospital and I have a large collection of images about my patients. However, I don't know what's in them. I would like to use these images to develop a neural network which can automatically classify them, however since my images aren't labeled, they aren't sufficient to train a classifier.

I reach out to 10 partner hospitals which DO have annotated data. I hope to train my new classifier on their datasets so that I can automatically label my own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, I will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

1) Ask 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels) \\
2) Use 10 partner models to predict on my local dataset, generating 10 labels for each datapoint \\
3) Then, for each local data point (now with 10 labels), I will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. I will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint. \\
4) Finally, I will retrain a new model on the local dataset which now has labels. This will be our final "DP" model.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [0]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [44]:
preds = (np.random.rand(num_examples, num_teachers) * num_labels).astype(int)# fake predictions

print(preds.shape)
print(preds)

new_labels = list()

for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)
    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)


print('Noisy 10000 labels curated from other hospitals')
print(len(new_labels))

(10000, 10)
[[4 3 2 ... 1 1 8]
 [3 8 2 ... 2 9 4]
 [6 9 0 ... 3 9 7]
 ...
 [2 3 4 ... 0 8 2]
 [0 8 3 ... 2 8 0]
 [1 7 9 ... 6 9 8]]
Noisy 10000 labels curated from other hospitals
10000


## PATE
Epsilon cannot be extended by post-processing.
 *  Generate a dataset with epsilon sensivity and train a deep learning model on this dataset and verify that the sensitivity does not increase

In [0]:
# !pip install syft

In [52]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 2, 8, 9])
counts = np.bincount(labels, minlength=10)
print(counts)
query_result = np.argmax(counts)
print(query_result)

from syft.frameworks.torch.dp import pate

num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers


data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps

print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

preds[:,0:50] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

[0 0 1 1 0 0 1 0 1 6]
9
Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 11.756462732485105
Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881
Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843
